In [7]:
# 论文：Efficient MILP Modelings for Sboxes and Linear Layers of SPN ciphers


# 使用PRESENT的S-box
from sage.crypto.sboxes import PRESENT

PRESENT.difference_distribution_table()
# 使用RECTANGLE的S-box
# from sage.crypto.sbox import SBox
# sagemath没有RECTANGLE，因此自己写出S-box
# S = SBox([6,5,12,10,1,14,7,9,11,0,3,13,8,15,4,2])
# S-box是4bit输出和4bit输出，因此 n=4+4=8
n = 8
# 给出S-box的可行的差分路径的点，实际就是ddt中不为0的点的坐标，这里我直接用一个[0,256）十进制表示了，合理的应该是两个[0,15)的十进制表示
S_pos = set([(16)*diff+PRESENT(x)^^PRESENT(x^^diff) for diff in range((16)) for x in range(16)])

# 如果给出的是PRESENT的S-box，那么要将S改为PRESENT，即
# S_pos = set([(16)*diff+PRESENT(x)^^PRESENT(x^^diff) for diff in range((16)) for x in range(16)])

# 取补集
S_complement = set([16*x+y for x in range(16) for y in range(16)]).difference(S_pos)
# 存储S_pos中点的二进制形式
Set_points = []
# 存储S_complement的二进制形式
Set_impossible = []
# S_pos十进制转二进制
for s in S_pos:
    s_binary = s.bits()
    if len(s_binary) != n:
        for i in range(n-len(s_binary)):
            s_binary.append(0) 
    Set_points.append(s_binary)
# S_complement十进制转二进制
for s in S_complement:
    s_binary = s.bits()
    if len(s_binary) != n:
        for i in range(n-len(s_binary)):
            s_binary.append(0) 
    Set_impossible.append(s_binary)
# generating the convex hull corresponding to S_pos  
C_set = Polyhedron(vertices = Set_points)

# gives us an initial set of inequalities 
# 注意不等式用一个9bit向量表示，第一个比特是常数项，后8个才是系数，不等式大于等于0
old_set = []
for ineq in C_set.inequality_generator():
    old_set.append(list(ineq))

In [8]:
# Algorithm 1 Compute a set of inequalities from possible transitions.
# 大概10min跑出结果
for p in Set_points:
    PH_set = []
    for ineq in old_set:
        if ineq[0] + sum(ineq[i]*p[i-1] for i in range(1,len(ineq))) == 0:
            PH_set.append(ineq)
    for c_1_index in range(len(PH_set)):
        c_1 = list(PH_set[c_1_index])
        # 要求点p满足不等式的等号 对应Algorithm 1 line 5,不过这里设置k=2，所以要有c_1,c_2
        for c_2_index in range(c_1_index+1,len(PH_set)):
            c_2 = list(PH_set[c_2_index])
            # 记录在S_complement中的点，这些点是符合c_1和c_2这两个不等式的，
            Set_original_points_without_S = []
            for p_impossible in Set_impossible:
                if c_2[0] + sum(c_2[i]*p_impossible[i-1] for i in range(1,n+1)) >= 0 and c_1[0] + sum(c_1[i]*p_impossible[i-1] for i in range(1,n+1)) >= 0:
                    Set_original_points_without_S.append(p_impossible)
#                     print(Set_original_points_without_S)
            c_new = c_1 + c_2
            # 如果Set_original_points_without_S中的点被c_new排除，那么c_new要加入到old_set中，对应Algorithm 1 line 7 8
            for x in Set_original_points_without_S:
                if c_new[0] + sum(c_new[i]*x[i-1] for i in range(1,n+1)) < 0:
                    print(1)
                    if c_new not in old_set:
                        old_set.append(c_new)
len(old_set)  

327

In [161]:
# # Algorithm 1 Compute a set of inequalities from possible transitions.
# # 大概几个h，没有计算多长时间，我正在跑，文章也说了要 a few hours
# for p in Set_points:
#     for c_1_index in range(len(old_set)):
#         c_1 = list(old_set[c_1_index])
#         # 要求点p满足不等式的等号 对应Algorithm 1 line 5,不过这里设置k=3，所以要有c_1,c_2，c_3
#         if c_1[0] + sum(c_1[i]*p[i-1] for i in range(1,n+1)) == 0:
#             for c_2_index in range(c_1_index+1,len(old_set)):
#                 c_2 = list(old_set[c_2_index])
#                 # 同上
#                 if c_2[0] + sum(c_2[i]*p[i-1] for i in range(1,n+1)) == 0:
#                     for c_3_index in range(c_2_index+1,len(old_set)):
#                         c_3 = list(old_set[c_3_index])
#                         if c_3[0] + sum(c_3[i]*p[i-1] for i in range(1,n+1)) == 0:
#                             # 记录在S_complement中的点，这些点是符合c_1和c_2这两个不等式的，
#                             Set_original_points_without_S = []
#                             for p_impossible in Set_impossible:
#                                 if c_3[0] + sum(c_3[i]*p_impossible[i-1] for i in range(1,n+1)) >= 0 and c_2[0] + sum(c_2[i]*p_impossible[i-1] for i in range(1,n+1)) >= 0 and c_1[0] + sum(c_1[i]*p_impossible[i-1] for i in range(1,n+1)) >= 0:
#                                     Set_original_points_without_S.append(p_impossible)
#                             c_new = c_1 + c_2 + c_3
#                             # 如果Set_original_points_without_S中的点被c_new排除，那么c_new要加入到old_set中，对应Algorithm 1 line 7 8
#                             for x in Set_original_points_without_S:
#                                 if c_new[0] + sum(c_new[i]*x[i-1] for i in range(1,n+1)) < 0:
#                                     if c_new not in old_set:
#                                         old_set.append(c_new)
# len(old_set)  

267

In [6]:
# todo 的算法，是ST17a， 应用了MILP模型，
# 瞬间得到结果
todo_list = []
for p in Set_impossible:
    p_ineq = []
    for i in range(len(old_set)):
        ineq = old_set[i]
        if ineq[0] + sum(ineq[i]*p[i-1] for i in range(1,n+1)) < 0:
            p_ineq.append(i)
    todo_list.append(p_ineq)

p = MixedIntegerLinearProgram(maximization=False, solver = "GLPK")
# a[i]=0 代表算法得到的最小的不等式集合中没有old_set中第i个不等式，a[i]=1就是有
# 所以min sum_{0<=i<=size(old_set)} a[i]即可
a = p.new_variable(integer=True, nonnegative=True)
for i in range(len(old_set)):
    p.set_max(a[i],1)
for j in range(len(todo_list)):
    p_ineqs = todo_list[j]
    p.add_constraint(sum(a[i] for i in p_ineqs) >= 1)
p.set_objective(sum(a[i] for i in range(len(old_set))))
p.solve()

values = p.get_values(a)
counter = 0
for i in range(len(values)):
    if values[i] == 1:
        counter = counter + 1
        print(old_set[i])
print("We need ",counter,"inequalities to represent the ddt of s-box")


[6, 0, -1, 2, -1, -1, -2, -2, -1]
[8, -1, 1, -2, -2, -1, 2, -2, -3]
[8, -1, -2, -2, 1, -1, -2, 2, -3]
[3, 1, -1, 0, -1, 0, -1, -1, 1]
[0, 4, 3, 4, 1, -2, 3, -2, 1]
[6, 2, 1, -2, -2, -1, -2, 1, -1]
[0, -1, 2, 3, -1, 2, -1, 2, 2]
[0, 1, 4, 1, 4, 3, -2, -2, 0]
[2, -1, 0, 0, 0, -1, 1, 1, -1]
[7, -3, 3, -1, -1, 1, -2, -3, 3]
[7, -3, -1, -1, 3, 1, -3, -2, 3]
[0, -1, 2, 2, 2, -1, 3, 3, -1]
[1, -1, -1, 2, 1, 1, 1, -1, 1]
[0, 2, -1, 1, -1, 1, 2, 2, 0]
[6, 2, -2, -2, 1, -1, 1, -2, -1]
[0, 1, 1, -4, 1, 2, 3, 3, 2]
[0, 4, 1, 4, 3, -2, -2, 3, 1]
[8, -1, -4, -2, -4, -1, 2, 2, 3]
[10, -3, 3, -4, 1, -2, -3, -2, 1]
[9, 1, -3, -3, -1, -3, 2, -2, 1]
[7, -1, -2, 0, -2, 2, -1, -1, -2]
We need  21 inequalities to represent the ddt of s-box


In [194]:
len(old_set)

327